In [31]:
## Goal of this notebook is to scale through a cluster

(https://landsat.usgs.gov/landsat_acq)
Use the tools on this page to determine when the Landsat 7 and Landsat 8 satellites acquire data over your area of interest, and to view the paths that were acquired on any given day. 

Landsat satellites image the entire Earth every 16 days in an 8-day offset. Landsat 7 acquires data in descending (daytime) node, while Landsat 8 acquires data in both descending and occasional ascending (nighttime) node. 

The Landsat 8 satellite orbits the the Earth in a sun-synchronous, near-polar orbit, at an altitude of 705 km (438 mi), inclined at 98.2 degrees, and circles the Earth every 99 minutes. The satellite has a 16-day repeat cycle with an equatorial crossing time: 10:00 a.m. +/- 15 minutes.

Landsat satellites image the entire Earth every 16 days in an 8-day offset. Landsat 7 acquires data in descending (daytime) node, while Landsat 8 acquires data in both descending and occasional ascending (nighttime) node. 

Row refers to the latitudinal center line of a frame of imagery. As the satellite moves along its path, the observatory instruments are continuously scanning the terrain below.  These will be squares centered on the orbital path, but tilted clockwise when views on the UTM projection used for the distributed data.

In [41]:
url = 'http://landsat-pds.s3.amazonaws.com/c1/L8/227/065/LC08_L1TP_227065_20200608_20200626_01_T1/'
redband = url+'LC08_L1TP_227065_20200608_20200626_01_T1_B{}.TIF'.format(4)

In [42]:
!wget {redband}

--2020-09-03 02:52:24--  http://landsat-pds.s3.amazonaws.com/c1/L8/227/065/LC08_L1TP_227065_20200608_20200626_01_T1/LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF
Resolving landsat-pds.s3.amazonaws.com (landsat-pds.s3.amazonaws.com)... 52.218.217.179
Connecting to landsat-pds.s3.amazonaws.com (landsat-pds.s3.amazonaws.com)|52.218.217.179|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59222647 (56M) [image/tiff]
Saving to: ‘LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF.2’

LC08_L1TP_227065_20 100%[===================>]  56.48M  98.6MB/s    in 0.6s    

2020-09-03 02:52:25 (98.6 MB/s) - ‘LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF.2’ saved [59222647/59222647]



In [43]:
!gdalinfo LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF

Driver: GTiff/GeoTIFF
Files: LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF
Size is 7621, 7761
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 21N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 21N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-57,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
         

In [44]:
import gdal
from gdalconst import GA_ReadOnly

data = gdal.Open('LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF', GA_ReadOnly)
geoTransform = data.GetGeoTransform()
minx = geoTransform[0]
maxy = geoTransform[3]
maxx = minx + geoTransform[1] * data.RasterXSize
miny = maxy + geoTransform[5] * data.RasterYSize
print ([minx, miny, maxx, maxy])
data = None

[573285.0, -916515.0, 801915.0, -683685.0]


EPSG:4326
WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS

Reproject "EPSG",32621 to EPSG:4326

Lets find the bounding box for the region

In [45]:
from osgeo import gdal

input_raster=gdal.Open('LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF')
output_raster = "Reproj" + 'LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF'
gdal.Warp(output_raster,input_raster,dstSRS='EPSG:4326')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7fdead5ff060> >

In [46]:
data = gdal.Open('ReprojLC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF', GA_ReadOnly)
geoTransform = data.GetGeoTransform()
minx = geoTransform[0]
maxy = geoTransform[3]
maxx = minx + geoTransform[1] * data.RasterXSize
miny = maxy + geoTransform[5] * data.RasterYSize
print ([minx, miny, maxx, maxy])
data = None

[-56.33759064732078, -8.290860873734873, -54.2594726602711, -6.178270448812246]


If you have trouble getting the right scene to cover the area you want:

Go to https://landsat.usgs.gov/wrs-2-pathrow-latitudelongitude-converter  and enter the lat/long. 
Then note the path row, and when you get the Landsat data, insure you have the correct path and row, which are listed in the download table.
Use https://landsat.usgs.gov/landsat_acq tool, which will show the coverage for each data
Path/Row shapefiles and KML: https://www.usgs.gov/land-resources/nli/landsat/landsat-shapefiles-and-kml-files
KML file direct download:https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/atoms/files/WRS-2_bound_world_0.kml

In [3]:
#get list of images
import requests
from bs4 import BeautifulSoup
import re
URL = 'https://landsatonaws.com/L8/227/065/'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

In [13]:
# Use beautiful soup to extract all the links, and Tier 1 only
LevelOnes=[]
for link in soup.findAll('a', attrs={'href': re.compile("^/L8/227/065")}):
    #print(link.get('href'))
    href = link.get('href')
    if href.endswith("T1"):
        LevelOnes.append(href)

In [11]:
len(LevelOnes)

126

In [14]:
# Base links
LevelOnes

['/L8/227/065/LC08_L1TP_227065_20200811_20200822_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20200726_20200807_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20200710_20200721_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20200624_20200707_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20200608_20200626_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20200608_20200626_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20200523_20200607_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20200507_20200509_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20200405_20200410_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20200217_20200225_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20191215_20191226_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20191129_20191216_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20191028_20191114_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20191012_20191018_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20190926_20191017_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20190910_20190917_01_T1',
 '/L8/227/065/LC08_L1TP_227065_20190825_20190903_01_T1',
 '/L8/227/065/LC08_L1TP_227065_

In [20]:
# Lets add Band 4 (Red) only
base="s3://landsat-pds/c1"
redlinks=[]
for link in LevelOnes:
    print(link.split('/')[4])
    redband = base+link+"/"+link.split('/')[4]+'_B{}.TIF'.format(4)
    redlinks.append(redband)

LC08_L1TP_227065_20200811_20200822_01_T1
LC08_L1TP_227065_20200726_20200807_01_T1
LC08_L1TP_227065_20200710_20200721_01_T1
LC08_L1TP_227065_20200624_20200707_01_T1
LC08_L1TP_227065_20200608_20200626_01_T1
LC08_L1TP_227065_20200608_20200626_01_T1
LC08_L1TP_227065_20200523_20200607_01_T1
LC08_L1TP_227065_20200507_20200509_01_T1
LC08_L1TP_227065_20200405_20200410_01_T1
LC08_L1TP_227065_20200217_20200225_01_T1
LC08_L1TP_227065_20191215_20191226_01_T1
LC08_L1TP_227065_20191129_20191216_01_T1
LC08_L1TP_227065_20191028_20191114_01_T1
LC08_L1TP_227065_20191012_20191018_01_T1
LC08_L1TP_227065_20190926_20191017_01_T1
LC08_L1TP_227065_20190910_20190917_01_T1
LC08_L1TP_227065_20190825_20190903_01_T1
LC08_L1TP_227065_20190724_20190801_01_T1
LC08_L1TP_227065_20190708_20190719_01_T1
LC08_L1TP_227065_20190622_20190704_01_T1
LC08_L1TP_227065_20190606_20190619_01_T1
LC08_L1TP_227065_20190505_20190520_01_T1
LC08_L1TP_227065_20190505_20190520_01_T1
LC08_L1TP_227065_20190419_20190423_01_T1
LC08_L1TP_227065

In [21]:
redlinks

['s3://landsat-pds/c1/L8/227/065/LC08_L1TP_227065_20200811_20200822_01_T1/LC08_L1TP_227065_20200811_20200822_01_T1_B4.TIF',
 's3://landsat-pds/c1/L8/227/065/LC08_L1TP_227065_20200726_20200807_01_T1/LC08_L1TP_227065_20200726_20200807_01_T1_B4.TIF',
 's3://landsat-pds/c1/L8/227/065/LC08_L1TP_227065_20200710_20200721_01_T1/LC08_L1TP_227065_20200710_20200721_01_T1_B4.TIF',
 's3://landsat-pds/c1/L8/227/065/LC08_L1TP_227065_20200624_20200707_01_T1/LC08_L1TP_227065_20200624_20200707_01_T1_B4.TIF',
 's3://landsat-pds/c1/L8/227/065/LC08_L1TP_227065_20200608_20200626_01_T1/LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF',
 's3://landsat-pds/c1/L8/227/065/LC08_L1TP_227065_20200608_20200626_01_T1/LC08_L1TP_227065_20200608_20200626_01_T1_B4.TIF',
 's3://landsat-pds/c1/L8/227/065/LC08_L1TP_227065_20200523_20200607_01_T1/LC08_L1TP_227065_20200523_20200607_01_T1_B4.TIF',
 's3://landsat-pds/c1/L8/227/065/LC08_L1TP_227065_20200507_20200509_01_T1/LC08_L1TP_227065_20200507_20200509_01_T1_B4.TIF',
 's3://l

In [23]:
redlinks[0]

's3://landsat-pds/c1/L8/227/065/LC08_L1TP_227065_20200811_20200822_01_T1/LC08_L1TP_227065_20200811_20200822_01_T1_B4.TIF'

In [24]:
import rasterio
print('Landsat on AWS:')
with rasterio.open(redlinks[0]) as src:
    print(src.profile)

Landsat on AWS:
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 7621, 'height': 7761, 'count': 1, 'crs': CRS.from_epsg(32621), 'transform': Affine(30.0, 0.0, 575385.0,
       0.0, -30.0, -683685.0), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'deflate', 'interleave': 'band'}


In [32]:
import dask
import xarray as xa
@dask.delayed
def read_red(item):
    red = xa.open_rasterio(item, chunks={'band': 1, 'x': 1024, 'y': 1024})
    mean = red.mean(dim=['x', 'y'])
    return mean

We got 126 images

In [29]:
from dask_gateway import GatewayCluster
from distributed import Client

cluster = GatewayCluster()
cluster.adapt(minimum=2, maximum=20) #Keep a minimum of 2 workers, but allow for scaling up to 20 if there is RAM and CPU pressure
client = Client(cluster) #Make sure dask knows to use this cluster for computations
cluster

In [33]:
lazy_datasets = []
for item in redlinks:
    ds = read_red(item)
    lazy_datasets.append(ds)

datasets = dask.compute(*lazy_datasets)

In [37]:
# What we just calculated was mean value (RAW) DN for RED band

In [38]:
# TODO - Get the translation using MTL to do TOA

In [39]:
# Shut down computing resources 
client.close()
cluster.close()